In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
import warnings
import locale
import calendar
import chardet
import os
from tkinter import messagebox
from datetime import datetime
from unidecode import unidecode

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

rj_path = 'bases/gestiones/RJ'
clasa_path = 'bases/gestiones/CLASA'
mornese_path = 'bases/gestiones/MORNESE'

rj_archivos = [file for file in os.listdir(rj_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.xlsx')]
clasa_archivos = [file for file in os.listdir(clasa_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.txt')]
mornese_archivos = [file for file in os.listdir(mornese_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.xlsx')]

print(rj_archivos)
print(clasa_archivos)
print(mornese_archivos)

In [5]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [6]:
def detectar_codificacion(ruta_archivo):
    with open(ruta_archivo, 'rb') as f:
        resultado = chardet.detect(f.read())
    return resultado['encoding']

In [ ]:
import rarfile

def descomprimir_rar(archivo_rar, directorio_destino):
    if not os.path.exists(archivo_rar):
        print(f"El archivo {archivo_rar} no existe.")
        return
    
    if not os.path.exists(directorio_destino):
        os.makedirs(directorio_destino)
    
    with rarfile.RarFile(archivo_rar) as archivo:
        archivo.extractall(path=directorio_destino)
        print(f"Archivos extraídos en {directorio_destino}")

descomprimir_rar('JUDICIAL.rar', 'bases/')

In [4]:
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    if 'AGENCIA' in df.columns:
        df['AGENCIA'] = df['AGENCIA'].str.upper().replace('ASESCOM RJ ABOGADOS', 'ASESCOM RJ')
    if 'TIPO_DE_MARCACIÓN' in df.columns:
        df.rename(columns={'TIPO_DE_MARCACIÓN': 'TIPO_MARCADOR'}, inplace=True)
    if 'TIPO_DE_DISCADO' in df.columns:
        df.rename(columns={'TIPO_DE_DISCADO': 'TIPO_MARCADOR'}, inplace=True)
    if 'PRIORIDAD_TELEFONO' not in df.columns:
        df['PRIORIDAD_TELEFONO'] = np.nan
    
    df['MARCA'] = df['MARCA'].str.strip().str.upper().replace(' ', '', regex=True)
    df['MARCA'] = np.where(df['MARCA'] == 'KST', 'KSTBC', df['MARCA'])
    df = df[df['MARCA'].isin(['BC', 'EXJ', 'KSTBC'])]
    df['FECHA_LLAMADA'] = pd.to_datetime(df['FECHA_LLAMADA'], errors='coerce')
    df['HORA_INICIO_LLAMADA'] = pd.to_datetime(df['HORA_INICIO_LLAMADA']).dt.hour.astype('Int64').fillna(0)
    df['TIPO_CONTACTO'] = df['TIPO_CONTACTO'].str.upper().replace('TERCERO', 'CONTACTO INDIRECTO').replace('CONTACTO EFECTIVO', 'CONTACTO DIRECTO').replace('EFECTIVO', 'CONTACTO DIRECTO')
    df['TRAMO'] = df['TRAMO'].str.strip().str.upper().replace(' ', '', regex=True).replace(['T1', 'T2', 'T3', 'T4'], 'NULL').fillna('NULL')
    df['PDP'] = df.apply(lambda x: 1 if pd.notna(x['FECHA_PDP']) else 0, axis=1)
    df['TIPO_MARCADOR'] = df['TIPO_MARCADOR'].str.strip().str.upper().replace(' ', '', regex=True).fillna('NULL')
    
    df = df[['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'FECHA_LLAMADA', 'HORA_INICIO_LLAMADA', 'TIPO_CONTACTO', 'TRAMO', 'PDP', 'TIPO_MARCADOR', 'PRIORIDAD_TELEFONO']]
    return df

In [5]:
def consolidar_archivos(agencia_path: str, archivos: list) -> pd.DataFrame:
    dataframes = []
    for archivo in archivos:
        ruta_completa = os.path.join(agencia_path, archivo)
        if archivo.endswith('.txt'):
            df = pd.read_csv(
                ruta_completa, 
                sep=',', 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}, 
                encoding='ISO-8859-1'
                )
        else:
            df = pd.read_excel(
                ruta_completa, 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}
                )
        
        df.columns = clean_columns(df.columns)
        dataframes.append(df)
        print(f'{archivo} leído correctamente')
    
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    df_consolidado = clean_dataframe(df_consolidado)
    
    df_consolidado.to_csv(
        os.path.join(agencia_path, f'CONSOLIDADO_GESTIONES_{agencia_path.split("/")[-1]}.csv'), 
        index=False, 
        sep='|'
        )
    return df_consolidado

In [6]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Consolidar archivos de gestiones?', icon='warning')

if result == 'yes':
    dataframes = {}
    archivos = {
        #'RJ': (rj_path, rj_archivos),
        'CLASA': (clasa_path, clasa_archivos),
        #'MORNESE': (mornese_path, mornese_archivos)
    }
    
    for key, (path, file_list) in archivos.items():
        if file_list:
            dataframes[key] = consolidar_archivos(path, file_list)
            print(f'Consolidado {key}:', dataframes[key].shape)
        else:
            print(f'No hay archivos para consolidar en {key}.')

root.destroy()

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Leer consolidados?', icon='warning')

if result == 'yes':
    dataframes = {}
    archivos = {
        #'RJ': f'{rj_path}/CONSOLIDADO_GESTIONES_RJ.csv',
        'CLASA': f'{clasa_path}/CONSOLIDADO_GESTIONES_CLASA.csv',
        #'MORNESE': f'{mornese_path}/CONSOLIDADO_GESTIONES_MORNESE.csv'
    }
    
    for key, file_path in archivos.items():
        if os.path.exists(file_path):
            dataframes[key] = pd.read_csv(
                file_path, 
                sep='|', 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}, 
                encoding='ISO-8859-1'
                )
            print(f'Consolidado {key} leído:', dataframes[key].shape)
        else:
            print(f'El archivo CONSOLIDADO_GESTIONES_{key}.csv no existe.')

root.destroy()

In [ ]:
def sort_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = ['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'TIPO_CONTACTO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR', 'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'CONTACTO_DIRECTO', 'PDP']
    return df[cols]

df_clasa_test: pd.DataFrame = dataframes['CLASA'].copy()
df_clasa_test.sort_values(by='FECHA_LLAMADA', inplace=True)
df_clasa_test.reset_index(inplace=True, drop=True)
df_clasa_test.drop(columns={'HORA_INICIO_LLAMADA'}, inplace=True)
df_clasa_test['TRAMO'] = df_clasa_test['TRAMO'].fillna('NULL')
print(df_clasa_test.columns)
df_clasa_test.head(5)

In [ ]:
df_clasa_test['PERIODO'] = pd.to_datetime(df_clasa_test['FECHA_LLAMADA']).dt.strftime('%Y%m').astype(str)

fechas_unicas = df_clasa_test[['PERIODO', 'FECHA_LLAMADA']].drop_duplicates().sort_values(by='FECHA_LLAMADA')
orden_fechas = fechas_unicas.groupby('PERIODO').cumcount() + 1
fechas_unicas['DIA'] = orden_fechas
df_clasa_test = df_clasa_test.merge(fechas_unicas[['PERIODO', 'FECHA_LLAMADA', 'DIA']], on=['PERIODO', 'FECHA_LLAMADA'], how='left')
df_clasa_test.drop(columns={'FECHA_LLAMADA'}, inplace=True)

df_clasa_test['INTENSIDAD_TOTAL'] = 1
df_clasa_test['INTENSIDAD_DIRECTA'] = np.where(df_clasa_test['TIPO_CONTACTO'] == 'CONTACTO DIRECTO', 1, 0)
df_clasa_test['CONTACTO_DIRECTO'] = np.where(df_clasa_test['INTENSIDAD_DIRECTA'] != 0, 1, 0)

df_clasa_test.sort_values(by=['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR'], inplace=True)
df_clasa_test.reset_index(drop=True, inplace=True)

df_clasa_test = df_clasa_test[df_clasa_test['PERIODO'] == '202411']
df_clasa_test

In [ ]:
# asignacion
#df_asig = pd.read_excel('bases/asignacion/202411/base_asignacion_NOV24.xlsx', sheet_name='BASE', dtype={'CONTRATO': str, 'COD_CENTRAL': str})
#df_asig = df_asig[df_asig['AGENCIA'] == 'CLASA MORA']
#df_asig['KEY'] = 'ASIGNACION'
#df_asig.shape

In [645]:
#df_asig.rename(columns={
#    'CC': 'COD_CENTRAL', 
#    'CARTERA': 'MARCA', 
#}, inplace=True)

#df_asig = df_asig[['COD_CENTRAL', 'CONTRATO','MARCA', 'AGENCIA', 'KEY']]
#df_asig['COD_CENTRAL'] = df_asig['COD_CENTRAL'].astype('Int64').astype(str).str.zfill(8)

In [684]:
#df_cruce = df_asig.merge(df_clasa_test, on='CONTRATO', how='left', suffixes=('', '_CLASA'))

#df_cruce.drop(columns={
#    'AGENCIA_CLASA',
#    'COD_CENTRAL_CLASA', 
#    'MARCA_CLASA', 
#}, inplace=True)

#df_cruce['TIPO_CONTACTO'] = df_cruce['TIPO_CONTACTO'].fillna('SIN GESTION')
#df_cruce['TRAMO'] = df_cruce['TRAMO'].fillna('NULL')
#df_cruce['PDP'] = df_cruce['PDP'].fillna(0)
#df_cruce['TIPO_MARCADOR'] = df_cruce['TIPO_MARCADOR'].fillna('SIN GESTION')
#df_cruce['PRIORIDAD_TELEFONO'] = df_cruce['PRIORIDAD_TELEFONO'].fillna('NULL')
#df_cruce['PERIODO'] = '202411'
#df_cruce['DIA'] = df_cruce['DIA'].fillna('NULL')

In [ ]:
#df_cruce['INTENSIDAD_TOTAL'] = np.where(df_cruce['INTENSIDAD_TOTAL'] == 1, 1, 0)
#df_cruce['INTENSIDAD_DIRECTA'] = np.where(df_cruce['TIPO_CONTACTO'] == 'CONTACTO DIRECTO', 1, 0)
#df_cruce['CONTACTO_DIRECTO'] = np.where(df_cruce['INTENSIDAD_DIRECTA'] != 0, 1, 0)

#df_cruce.sort_values(by=['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR'], inplace=True)
#df_cruce.reset_index(drop=True, inplace=True)
#df_cruce

In [ ]:
df_clasa_test = sort_columns(df_clasa_test)
df_clasa_test

In [ ]:
df_clasa_test_5 = df_clasa_test[df_clasa_test['DIA'].isin([1,2,3,4,5])]
df_clasa_test_5['DIA'].value_counts(dropna=False)

In [ ]:
dias_unicos = sorted(df_clasa_test_5['DIA'].unique())
dias_unicos

In [ ]:
df_test_groupped_5 = pd.DataFrame()

for dia in dias_unicos:
    df_acumulado = df_clasa_test[df_clasa_test['DIA'] <= dia]
    print('Dia acumulado:', dia, df_acumulado.shape, df_acumulado['DIA'].unique())
    
    df_grouped = df_acumulado.groupby(['AGENCIA', 'PERIODO', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR']).agg({
        'INTENSIDAD_TOTAL': 'sum', 
        'INTENSIDAD_DIRECTA': 'sum', 
        'CONTACTO_DIRECTO': 'max', 
        'PDP': 'max'
    }).reset_index()
    
    df_grouped['TASA_CIERRE'] = np.where(
        (df_grouped['INTENSIDAD_DIRECTA'] > 0) & (df_grouped['PDP'] == 1), 
        1, 
        np.where(
            (df_grouped['INTENSIDAD_DIRECTA'] > 0) & (df_grouped['PDP'] == 0), 
            0, 
            np.nan
            )
        )
    df_grouped['TASA_CIERRE'] = df_grouped['TASA_CIERRE'].apply(lambda x: int(x) if pd.notna(x) else x)
    
    df_grouped['DIA'] = dia
    
    df_test_groupped_5 = pd.concat([df_test_groupped_5, df_grouped])

df_test_groupped_5.sort_values(by=['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR'], inplace=True)
df_test_groupped_5.reset_index(drop=True, inplace=True)

In [ ]:
def sort_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = ['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR', 'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'CONTACTO_DIRECTO', 'PDP']
    return df[cols]

df_test_groupped_5 = sort_columns(df_test_groupped_5)
df_test_groupped_5

In [77]:
df_test_groupped_5.to_excel('bases/gestiones/CLASA/RESUMEN_5_DIAS_CLASA.xlsx', index=False)
os.startfile('C://Users//p042833//Documents//GitHub//base-pagos//bases//gestiones//CLASA//RESUMEN_5_DIAS_CLASA.xlsx')

In [ ]:
df_clasa_test.isnull().sum()

In [ ]:
df_clasa_test['DIA'].value_counts(dropna=False).sort_index()

In [84]:
def sort_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = ['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR', 'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'CONTACTO_DIRECTO', 'TASA_CIERRE']
    return df[cols]

In [ ]:
dias_unicos = sorted(df_clasa_test['DIA'].unique())
dias_unicos

In [ ]:
df_test_groupped = pd.DataFrame()

for dia in dias_unicos:
    df_acumulado = df_clasa_test[df_clasa_test['DIA'] <= dia]
    print('Dia acumulado:', dia, df_acumulado.shape, df_acumulado['DIA'].unique())
    
    df_grouped = df_acumulado.groupby(['AGENCIA', 'PERIODO', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR']).agg({
        'INTENSIDAD_TOTAL': 'sum', 
        'INTENSIDAD_DIRECTA': 'sum', 
        'CONTACTO_DIRECTO': 'max', 
        'PDP': 'max'
    }).reset_index()
    
    df_grouped['TASA_CIERRE'] = np.where(
        (df_grouped['INTENSIDAD_DIRECTA'] > 0) & (df_grouped['PDP'] == 1), 
        1, 
        np.where(
            (df_grouped['INTENSIDAD_DIRECTA'] > 0) & (df_grouped['PDP'] == 0), 
            0, 
            np.nan
            )
        )
    df_grouped['TASA_CIERRE'] = df_grouped['TASA_CIERRE'].apply(lambda x: int(x) if pd.notna(x) else x)
    
    df_grouped = df_grouped.groupby(['AGENCIA', 'PERIODO', 'MARCA', 'TRAMO', 'TIPO_MARCADOR', 'PRIORIDAD_TELEFONO']).agg({
        'INTENSIDAD_TOTAL': 'mean', 
        'INTENSIDAD_DIRECTA': lambda x: x[x != 0].mean(), 
        'CONTACTO_DIRECTO': lambda x: x.mean(skipna=True), 
        'TASA_CIERRE': lambda x: x.mean(skipna=True), 
    }).reset_index()
    
    df_grouped['INTENSIDAD_TOTAL'] = df_grouped['INTENSIDAD_TOTAL'].round(2)
    df_grouped['INTENSIDAD_DIRECTA'] = df_grouped['INTENSIDAD_DIRECTA'].round(2)
    df_grouped['CONTACTO_DIRECTO'] = df_grouped['CONTACTO_DIRECTO'].round(2)
    df_grouped['TASA_CIERRE'] = df_grouped['TASA_CIERRE'].round(2)
    df_grouped['DIA'] = dia
    
    df_test_groupped = pd.concat([df_test_groupped, df_grouped])

df_test_groupped.sort_values(by=['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR'], inplace=True)
df_test_groupped.reset_index(drop=True, inplace=True)
df_test_groupped.shape

In [ ]:
df_test_groupped = sort_columns(df_test_groupped)
df_test_groupped

In [ ]:
df_test_groupped.isnull().sum()

In [89]:
df_test_groupped.fillna(0, inplace=True)

In [ ]:
df_test_groupped.isnull().sum()

In [ ]:
df_test_groupped['DIA'].value_counts(dropna=False).sort_index()

In [69]:
df_test_groupped.to_excel('bases/gestiones/CLASA/PRUEBA.xlsx', index=False)
os.startfile('C://Users//p042833//Documents//GitHub//base-pagos//bases//gestiones//CLASA//PRUEBA.xlsx')